In [4]:
import numpy as np
import matplotlib.pyplot as plt
from parse import parse
import cv2

In [5]:
train_images, train_labels = parse('facedata/facedatatrain', 'facedata/facedatatrainlabels', True)
validation_images, validation_labels = parse('facedata/facedatavalidation', 'facedata/facedatavalidationlabels', True)
test_images, test_labels = parse('facedata/facedatatest', 'facedata/facedatatestlabels', True)

In [6]:
base_rate_true = np.bincount(train_labels)/train_labels.shape[0]
print(base_rate_true)

[0.51884701 0.48115299]


In [11]:
kernel = np.ones((7,3))
images = np.swapaxes(np.swapaxes(train_images.astype(np.float64), 0, 2), 0, 1)
dst = cv2.filter2D(src = images, ddepth = -1, kernel = kernel, anchor = (0,0))
zones = dst[::7,::3,:]

vimages = np.swapaxes(np.swapaxes(validation_images.astype(np.float64), 0, 2), 0, 1)
vdst = cv2.filter2D(src = vimages, ddepth = -1, kernel = kernel, anchor = (0,0))
vzones = vdst[::7,::3,:]

In [98]:

for k in range(5000):
    for i in range(0, 450):
        arr = zones[:,:,i].ravel()
        b = np.dot(weights, arr) +bias
        if b < 0 and train_labels[i] == True:
            weights += arr
            bias += 1
        elif b >= 0 and train_labels[i] == False:
            weights -= arr
            bias -= 1

        


    

    

In [7]:
def train_perceptron_faces(passes, input, labels, zones):
    weights = np.zeros(200)
    bias = 0
    for k in range(passes):
        for i in range(0, len(input)):
            arr = zones[:,:,i].ravel()
            b = np.dot(weights, arr) +bias
            if b < 0 and labels[i] == True:
                weights += arr
                bias += 1
            elif b >= 0 and labels[i] == False:
                weights -= arr
                bias -= 1
    return weights, bias


In [2]:
def run_perceptron_faces(weights, input, labels, zones, bias):
    tally = 0
    for i in range(len(input)):
        arr = zones[:,:,i].ravel()
        b = np.dot(weights, arr) + bias 
        if (b >= 0 and labels[i] == True) or (b < 0 and labels[i] == False):
            tally += 1
    print(tally/len(input))


In [12]:
w, b = train_perceptron_faces(5000, train_images, train_labels, zones)

run_perceptron_faces(w, validation_images, validation_labels, vzones, b)

0.9003322259136213
